### 创建石墨烯文件
1. 构建石墨烯原胞基矢     
2. 得到扩胞后基矢的向量     
3. 平移得到扩胞后周期性原子的分数坐标   
4. 将分数坐标转成笛卡尔坐标       
6. 写入lammps的结构data文件      
7. 记录不同晶格位置对应的原子ID，写入map.in 文件*     
8. 运行LAMMPS* ， lmp - in in.graphene     

### 扩胞后原子坐标和晶格信息    
1. 构建石墨烯原胞的基矢   
2. 原胞坐标 $\frac{frac+(i-1)}{N}$ , N是扩胞大小，$1\leq i \leq n$     
3. 原胞基矢点乘
$$
	\begin{bmatrix}
	N_{1} & 0 \\
	0 & N_{2} 
	\end{bmatrix} 
$$ 		
&emsp;&emsp; $N_1 \ N_2$ 分别为x ，y方向扩胞的大小。







### 构建基矢

In [19]:
import math
import numpy as np
a = (2.522,0,0)
b = (2.522/2,2.522/2*math.sqrt(3),0)
c = (0,0,20)
print(a,b,c)

(2.522, 0, 0) (1.261, 2.184116068344354, 0) (0, 0, 20)


### 扩胞

In [20]:
super_x = 10
super_y = 10
super_z = 1

In [21]:
#变换矩阵
tranformation = np.array([[super_x,0,0],[0,super_y,0],[0,0,super_z]])
print(tranformation)

[[10  0  0]
 [ 0 10  0]
 [ 0  0  1]]


### 扩胞后的矩阵

In [22]:
lattice = np.array([a,b,c])
print(lattice)

[[ 2.522       0.          0.        ]
 [ 1.261       2.18411607  0.        ]
 [ 0.          0.         20.        ]]


In [23]:
#扩胞之后的基矢
new_lattice = np.dot(lattice,tranformation)
print(new_lattice)

[[25.22        0.          0.        ]
 [12.61       21.84116068  0.        ]
 [ 0.          0.         20.        ]]


In [24]:
# 原来原胞里面的基矢
C1 = [0,0,0.5]
C2 = [1/3,1/3,0.5]
print(C1,C2)

[0, 0, 0.5] [0.3333333333333333, 0.3333333333333333, 0.5]


### 新构建的原子坐标

In [25]:
all_atoms = []
for i in range(super_x):
	for j in range(super_y):
		newC1 = [(0+i)/super_x,(0+j)/super_y,0.5]
		all_atoms.append(newC1)
		newC2 = [(1/3+i)/super_x,(1/3+j)/super_y,0.5]
		#print(newC2)
		all_atoms.append(newC2)
print(len(all_atoms))

200


### 把分数坐标改成笛卡尔坐标

np.array(all_atoms)

In [26]:
new_all_atoms = np.dot(np.array(all_atoms),new_lattice)
print(new_all_atoms)

[[ 0.          0.         10.        ]
 [ 1.261       0.72803869 10.        ]
 [ 1.261       2.18411607 10.        ]
 [ 2.522       2.91215476 10.        ]
 [ 2.522       4.36823214 10.        ]
 [ 3.783       5.09627083 10.        ]
 [ 3.783       6.55234821 10.        ]
 [ 5.044       7.28038689 10.        ]
 [ 5.044       8.73646427 10.        ]
 [ 6.305       9.46450296 10.        ]
 [ 6.305      10.92058034 10.        ]
 [ 7.566      11.64861903 10.        ]
 [ 7.566      13.10469641 10.        ]
 [ 8.827      13.8327351  10.        ]
 [ 8.827      15.28881248 10.        ]
 [10.088      16.01685117 10.        ]
 [10.088      17.47292855 10.        ]
 [11.349      18.20096724 10.        ]
 [11.349      19.65704462 10.        ]
 [12.61       20.3850833  10.        ]
 [ 2.522       0.         10.        ]
 [ 3.783       0.72803869 10.        ]
 [ 3.783       2.18411607 10.        ]
 [ 5.044       2.91215476 10.        ]
 [ 5.044       4.36823214 10.        ]
 [ 6.305       5.09627083

### 写入lammp.data 文件

In [27]:
print(new_lattice)

[[25.22        0.          0.        ]
 [12.61       21.84116068  0.        ]
 [ 0.          0.         20.        ]]


In [38]:
with open("Class-6.data",'w') as writer:
	writer.write('Daren\n')   #第一行title
	writer.write('\n')   #第一行title
	writer.write('%d atoms\n' %(len(all_atoms)))   #第二行原子总数目
	writer.write('1 atom types\n')   #原子类型，这里面只有一种原子类型
	writer.write('\n')   
	#晶格矢量
	writer.write('%7.3f %7.3f  xlo xhi\n' %(0.0,new_lattice[0][0]))   #xlo从0开始第二个是xx
	writer.write('%7.3f %7.3f  ylo yhi\n' %(0.0,new_lattice[1][1]))  #ylo:0 yhi:yy
	writer.write('%7.3f %7.3f  zlo zhi\n' %(0.0,new_lattice[2][2]))  #zlo:0 zhi:zz
	writer.write('%7.3f %7.3f %7.3f xy xz yz\n' %(new_lattice[1][0],new_lattice[2][0],new_lattice[2][1]))  #写b在x方向偏向的矢量 xy ，xz，yz
	# 写入原子信息
	writer.write('\n') 
	writer.write('Masses\n') 
	writer.write('\n')  	
	writer.write('1 12.0107\n') 
	writer.write('\n') 
	writer.write('Atoms\n') 
	writer.write('\n') 
	# 把new_all_atoms分别写入
	index = 1#写入原子的id 第几个原子
	for i in new_all_atoms:
		writer.write('%d 1 %7.3f %7.3f %7.3f\n' %(index,i[0],i[1],i[2]))
		index = index + 1

In [36]:
print(new_all_atoms)

[[ 0.          0.         10.        ]
 [ 1.261       0.72803869 10.        ]
 [ 1.261       2.18411607 10.        ]
 [ 2.522       2.91215476 10.        ]
 [ 2.522       4.36823214 10.        ]
 [ 3.783       5.09627083 10.        ]
 [ 3.783       6.55234821 10.        ]
 [ 5.044       7.28038689 10.        ]
 [ 5.044       8.73646427 10.        ]
 [ 6.305       9.46450296 10.        ]
 [ 6.305      10.92058034 10.        ]
 [ 7.566      11.64861903 10.        ]
 [ 7.566      13.10469641 10.        ]
 [ 8.827      13.8327351  10.        ]
 [ 8.827      15.28881248 10.        ]
 [10.088      16.01685117 10.        ]
 [10.088      17.47292855 10.        ]
 [11.349      18.20096724 10.        ]
 [11.349      19.65704462 10.        ]
 [12.61       20.3850833  10.        ]
 [ 2.522       0.         10.        ]
 [ 3.783       0.72803869 10.        ]
 [ 3.783       2.18411607 10.        ]
 [ 5.044       2.91215476 10.        ]
 [ 5.044       4.36823214 10.        ]
 [ 6.305       5.09627083